# Python with Firebase 
- https://medium.com/faun/getting-started-with-firebase-cloud-firestore-using-python-c6ab3f5ecae0
- https://towardsdatascience.com/nosql-on-the-cloud-with-python-55a1383752fc for firestore


In [1]:
# dependencies
# !pip install firebase-admin

In [2]:
import firebase_admin
import dateutil
from datetime import datetime as dt

from firebase_admin import credentials,firestore
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)
firestore_db = firestore.client() # mamke complete connection

In [3]:
import pandas as pd
import os

In [98]:
# Load Document

## load Student Info
df = pd.read_csv("../src/assets/studentInfo.csv",header='infer')
dfSubjectCode = pd.read_csv("../src/assets/subjectCode.csv",header='infer')

In [99]:
df.columns = ['name', 'middleName', 'fatherName', 'sex', 'cast', 'handicap', 'SUB1', 'SUB2',
       'MED2', 'SUB3', 'MED3', 'SUB4', 'MED4', 'SUB5', 'MED5', 'SUB6', 'MED6',
       'SUB7', 'MED7', 'DOB', 'ANNUAL_INC', 'ONLY_CHILD', 'ADM_SRL',
       'ADM_DATE', 'MINORITY', 'AADHAR']

In [100]:
# preprocess dataframe
df = df.fillna('null')
df['DOB'] = [dt.strptime(fixDate(x),"%d%m%Y") for x in df['DOB']]
df['ADM_DATE'] = [dt.strptime(fixDate(x),"%d%m%Y") for x in df['ADM_DATE']]

In [91]:
# important Funcitons
def parseDic(a):
    for i in a:
        if str(a[i]).isnumeric():
            a[i] = int(a[i])
    return a

def fixDate(s):
    s = str(s)
    if len(s) < 8:
        return "0"+s
    return s

In [93]:
# Push data in firestore
col = df.columns
fb = firestore_db.collection('studentInfo')
for i in range(len(df)):
    d = df.iloc[i]
    admnNo = d["ADM_SRL"]
    dic = { str(z):d[z] for z in col if z != "ADM_SRL"}
    dic = parseDic(dic)
    fb.document(str(admnNo)).set(dic)
    

In [101]:
dfSubjectCode.head()

,code,subjectName
0,2,HINDI COURSE-A
1,3,URDU COURSE-A
2,4,PUNJABI
3,5,BENGALI
4,6,TAMIL


In [104]:
# Push data in firestore

def parseCode(a):
    l = 3
    an = len(a)
    return '0'*(l-an)+a

fb = firestore_db.collection('subjectCode')
for i in range(len(dfSubjectCode)):
    d = dfSubjectCode.iloc[i]
    primaryKey = d["code"]
    fb.document(parseCode(str(primaryKey))).set({'name':d["subjectName"]})
    

In [6]:
col1 = firestore_db.collection('course') # Tree Structure
 
doc1 = col1.document("English")
doc2 = col1.document("Hindi")

doc1.set({'teacher':'nazia','phon':3243}) # Set the Data of In firebase
doc2.set({'teacher':'Hemlata','phon':243324})
doc2.update({'dummy':'haah'}) # update data add new key value pair
doc2.update({'dummy':firestore.ArrayUnion(["marks"])}) # similarly ArrayRemove

# doc1.delete() # delete the node

# collection.document('barcelona').update({'weather': firestore.DELETE_FIELD})
# delete Single Field

col1.where('class','>',4).get()

In [38]:
v = firestore_db.collection('studentInfo').document('MIS001').collection('name').stream()

In [43]:
print(v.iter)

AttributeError: 'generator' object has no attribute 'iter'

In [ ]:
firestore_db.collection('course').add({'song': 'Imagine', 'artist': 'John Lennon'})